<a href="https://colab.research.google.com/github/RafaelNovais/MasterAI/blob/master/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Rafael Novais de Melo -
23113607**

### Problem definition

A company ABC have 1238 employees. ABC wants to buy a new insurance policy for its employees. Attach is a dataset (train-2.csv Download train-2.csv) about some of their previous employees where the company paid medical costs billed by a health insurance company XYZ. The total cost was 16410282.7 euros.

ABC recently recruited about 100 additional employees. XYZ gave them a high price (i.e. 17755825 euros) for the same policy considering there will be additional 100 employees.

You are working in ABC as a data analyst/research engineer. Your boss asked you to rapidly use the existing data that you have to see whether the given price is reasonable or not. Can you accurately predict the costs for the new employees based on their given information? Report the total price (17755825 euros) that is the previous total price (16410282.7) plus the new predicted price for the new employees with specific features given in the testing set (test-3.csv Download test-3.csv).

Do the following:

First, report what type of algorithm you need to use and why? [1 Mark]
Use all the techniques of pre-processing, cleaning, and normalisation to prepare your data. Report each with justification why you want to use. [3 Marks]
Analyse the data by visualising each feature or overall whole dataset with various graphs, bars, etc. to understand the data before applying a model. Report what you learned from visualising the data. Did you find any correlation or discrepancies in the data. [3 Marks]
Use the k-fold cross validation approach to find the optimal model that you can apply on the testing data provided. Report the performance e.g loss and loss curve,  validation accuracy and its curve (if applicable) and show when and why you stopped e.g., did you reached convergence or not. [2.75 Marks]
Report the final total price that you got. [0.25 Marks]
This time you need to apply and present the results in the best way possible to your boss. Considering you have learned various ways.

Upload your pdf of not more than 4 pages in the area designated for pdf upload.

In addition, upload the code in the designated area for code upload. Plagiarism will be strictly penalised.

In [58]:
#import Liberies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from sklearn.base import BaseEstimator
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
import seaborn as sns
from scipy.spatial import ConvexHull
import requests
from io import StringIO
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline




In [ ]:

#Read csv file to train and test
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train-2.csv')
#df = upload_file()


#Preparing the Data
cleaned_df = clean_df(df)


#Select the Target Colunm name
target_column_name =  select_target(cleaned_df)


#Visualization and Graphioc Analise
analyze_data(cleaned_df, target_column_name, categorical_cols)


#Select data and split in Train and Test
x_train, x_test, y_train, y_test  = split_data(cleaned_df, target_column_name)


#I Couldn't use Kfold with encode the categorical variable to predict so i return split_test_train
#kfold_splits = split_data_kfold(df_encode, target_column_name)


#Encode the categorical variables
categorical_cols = select_cat(x_train)
x_train_encode = encode_cat(x_train,categorical_cols)
x_test_encode = encode_cat(x_test,categorical_cols)


#Classification
y_predict_rfr = class_random_forest_regressor(x_train_encode, y_train, x_test_encode)


#Verifiy is the metrics from the classification
r2_rfr = metrics(y_test,y_predict_rfr)
total_y_predict_rfr = sum(y_predict_rfr)
total_y_test = sum(y_test)
print('Metrics from Test')
print(total_y_predict_rfr,total_y_test,r2_rfr)

#Read CSV file from the new employees
new_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test-3.csv')
#new_df = uploadfile()


#Preparing the Data from the new employees
x_new_df = new_df.drop(columns=target_column_name)
new_x_cleaned_df = clean_df(x_new_df)


#Encode the categorical variables Data from new employees
x_encode = encode_cat(new_x_cleaned_df,categorical_cols)


#Classification for the new employees
predict_new_employees_rfr = class_random_forest_regressor(x_train_encode, y_train, x_encode)


#Verifiy is the metrics from the classification for the new employees
total_new_employees = sum(predict_new_employees_rfr)
print('Total fom new Employees')
total_new_employees


#Compare High price and Predict Price
old_price = 16410282.7
new_price = 17755825
diff_price = new_price - old_price
print('Predict Price and High Price')
total_new_employees,diff_price
print('Difference between XYZ company is asking for the 100 employs and the amount we predict')
diff_price-total_new_employees











In [ ]:
#Testing Kfold with encode - DIDN'T Work
kfold_splits = split_data_kfold(cleaned_df, target_column_name)
all_predictions = []
for fold, (x_train, x_test, y_train, y_test) in enumerate(kfold_splits, start=1):
    # Assuming x_train and x_test are encoded (use x_train_encode, x_test_encode if needed)
    y_pred_fold = class_random_forest_regressor(x_train, y_train, x_test)
    all_predictions.append(y_pred_fold)
    total_all_prediction = sum(all_predictions)


    # Evaluate the predictions (you can use other metrics like mean squared error)
    r2 = r2_score(y_test, y_pred_fold)
    print(f'Fold {fold}: Mean Squared Error = {r2}')
    print('Total')
    total_all_prediction


In [3]:
#Upload File

def upload_file(): #Upload/Read csv file
    file_path = input("Please enter the file path or URL to upload: ")
    df = pd.read_csv(file_path)
    return df


In [4]:
#Clean the dataframe
def clean_df(data):
  cleaned_data = data.dropna()
  return cleaned_data


In [7]:
#Select Target column to y
def select_target(data):
    print("Columns in the DataFrame:")  # Show all column names to select 2
    for column in data.columns:
        print(column)

    column_name = input("Enter the name of the Target column: ")
    return column_name

    if column_name not in data.columns:
        raise ValueError("Invalid column name. Please enter a valid column name.")

In [8]:
#Create a data visualisation to identify Outliers
def outliers(data,column_name):

  z_scores = np.abs((data[column_name] - data[column_name].mean()) / data[column_name].std())
  outliers_df = data[z_scores > 3]


  plt.figure(figsize=(12, 8))
  sns.boxplot(x=data[column_name])
  plt.title(f'Boxplot of {column_name} with Outliers Highlighted')
  plt.scatter(x=outliers_df.index, y=outliers_df[column_name], color='red', label='Outliers')
  plt.xlabel('Index')
  plt.ylabel(column_name)
  plt.legend()
  plt.show()

  return outliers_df





In [36]:
#Split the data Using Kfold
def split_data_kfold(data, column_name):
    x_data = data.drop(columns=column_name)
    y_data = data[column_name]
    kf = KFold(n_splits=10, shuffle=True, random_state=10)
    splits = []
    X_encoded = encode_cat(x_data,column_name)

    for train_index, test_index in kf.split(X_encoded):
        x_train, x_test = x_data.iloc[train_index], x_data.iloc[test_index]
        y_train, y_test = y_data.iloc[train_index], y_data.iloc[test_index]

        splits.append((x_train, x_test, y_train, y_test))

    return splits




In [9]:
#Split the Data into Train/Test
def split_data(data,column_name):

    x_data = data.drop(columns=column_name)  # Drop columns specified in column_name from x_data
    y_data = data[column_name]


    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.10, random_state=10)

    return x_train, x_test, y_train, y_test


In [10]:
# Select the columns are categorical
def select_cat(x_data):
    print("Columns in the DataFrame:")
    for column in df.columns:
        print(column)

    categorical_cols_str = input("Enter the names of categorical columns separated by ,: ")
    categorical_cols = categorical_cols_str.split(',')
    return categorical_cols

In [11]:
# encode categorical variables into numeric representations
def encode_cat(x_data,categorical_cols):

    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(), categorical_cols)
        ],
        remainder='passthrough'
    )

    x_encoded = preprocessor.fit_transform(x_data)

    return x_encoded

In [12]:
#Random Forest Regressor Classification
def class_random_forest_regressor(x_train,y_train,x_test):
    model = RandomForestRegressor(n_estimators=200 ,random_state=0)
    model.fit(x_train,y_train)

    predict = model.predict(x_test)

    return predict



In [59]:
# Logistic Regression Classification
def class_logistic_regression(x_train, y_train, x_test):
  model = LogisticRegression()
  model.fit(x_train,y_train)

  predict = model.predict(x_test)

  return predict


In [13]:
#K-Fold Split and Validation to compare

def k_fold_cross_validation(data,column_name):

  x_data = data.drop(columns=column_name)  # Drop columns specified in column_name from x_data
  y_data = data[column_name]

  model = RandomForestRegressor()
  kf = KFold(n_splits=10, shuffle=True, random_state=0)
  losses = []

  for train_index, val_index in kf.split(x_data):
     X_train, X_val = x_data.iloc[train_index], x_data.iloc[val_index]
     y_train, y_val = y_data.iloc[train_index], y_data.iloc[val_index]
     model.fit(X_train, y_train)

     y_val_pred = model.predict(X_val)

     loss = mean_squared_error(y_val, y_val_pred)
     losses.append(loss)

  average_loss = np.mean(losses)
  return average_loss






In [14]:
#Model Evaluation Metrics Compare
def metrics(y_test,y_pred):

    r2 = r2_score(y_test, y_pred)

    return r2


In [62]:
#Visualization and Graphioc Analise
def analyze_data(cleaned_df, target_column_name, categorical_cols):

    # Correlation Analysis
    correlation_matrix = cleaned_df.corr()

    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Correlation Matrix')
    plt.show()

    # Pair Plots
    sns.pairplot(cleaned_df)
    plt.show()

    # Discrepancy Analysis



    # Value Counts for Categorical Variables
    for column in categorical_cols:
        print(f'Value Counts for {column}:')
        print(cleaned_df[column].value_counts())
        print('\n')

    # Box Plots for Numerical Variables
    for column in ['age', 'bmi', 'children']:
        plt.figure(figsize=(8, 6))
        sns.boxplot(x=cleaned_df[column])
        plt.title(f'Box Plot for {column}')
        plt.show()

    # Histograms for Numerical Variables
    cleaned_df.hist(figsize=(10, 8))
    plt.show()

    # Bar Plots for Categorical Variables
    for column in categorical_cols:
        plt.figure(figsize=(8, 6))
        cleaned_df[column].value_counts().plot(kind='bar')
        plt.title(f'Bar Plot for {column}')
        plt.show()



In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
